In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
import numpy as np
import matplotlib.colors as mcolors
from matplotlib.cm import ScalarMappable
from scipy.stats import spearmanr
import geopandas as gpd
from matplotlib.patches import PathPatch
from matplotlib.path import Path
from shapely.geometry import Polygon, MultiPolygon
import matplotlib.patches as patches
import matplotlib.lines as mlines

In [ ]:
data = pd.read_csv('GWS.csv')
lat_lon = data[['lat', 'lon']]
#lat_lon

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# Calculated the difference in groundwater storage between successive months in each year
data_diff = data.iloc[:, 2:].diff(axis=1)
data_diff

In [ ]:
Ave_Ann = data_diff.iloc[:, 9:].mean(axis=1)#This starts from column 10 which coincides with 2003

In [ ]:
Ave_Ann = pd.DataFrame(Ave_Ann)

Ave_Ann.columns = ['Ave']

In [ ]:
MonthlyStorageChange = pd.concat([lat_lon, Ave_Ann], axis=1)

In [ ]:
MonthlyStorageChange
MonthlyStorageChange.to_csv('MonthlyStorageChange.csv', index=None)

In [ ]:
stor_change = -2.427
MonthlyStorageChange_subset = MonthlyStorageChange[MonthlyStorageChange['Ave'] < stor_change]
MonthlyStorageChange_subset

In [ ]:
A, B = MonthlyStorageChange['Ave'].min(), MonthlyStorageChange['Ave'].max()
A, B

In [ ]:


# Function to add a scalebar
def add_scalebar(ax, length, location=(0.5, 0.05), linewidth=4):
    llx0, llx1, lly0, lly1 = ax.get_extent(ccrs.PlateCarree())
    sbllx = (llx1 + llx0) / 2
    sblly = lly0 + (lly1 - lly0) * location[1]
    tmc = ccrs.TransverseMercator(sbllx, sblly)
    x0, x1, y0, y1 = ax.get_extent(tmc)
    sbx = x0 + (x1 - x0) * location[0]
    sby = y0 + (y1 - y0) * location[1]
    bar_xs = [sbx - length * 500, sbx + length * 500]
    ax.plot(bar_xs, [sby, sby], transform=tmc, color='k', linewidth=linewidth)
    ax.text(sbx, sby, str(length) + ' km', transform=tmc,
            horizontalalignment='center', verticalalignment='bottom')

min_value = -2.427605896916667
max_value = 1.0563268152693623

# Get the absolute maximum value in your data
abs_max = max(abs(min_value), abs(max_value))

# Redefined min_value and max_value to be symmetric around zero
min_value = -abs_max
max_value = abs_max

interval = (max_value - min_value) / 10
bounds = np.linspace(min_value, max_value, 11)
neg_cmap = mcolors.LinearSegmentedColormap.from_list("neg_cmap", ["darkred", "yellow"], 5)
pos_cmap = mcolors.LinearSegmentedColormap.from_list("pos_cmap", ["lightblue", "darkblue"], 5)
cmap = mcolors.ListedColormap(np.concatenate((neg_cmap(np.linspace(0, 1, 5)), pos_cmap(np.linspace(0, 1, 5)))))
norm = mcolors.BoundaryNorm(bounds, cmap.N)

def get_color(value, bounds, colors):
    for i in range(len(bounds) - 1):
        if value >= bounds[i] and value < bounds[i + 1]:
            return colors[i]
    return colors[-1]

MonthlyStorageChange['color'] = MonthlyStorageChange['Ave'].apply(lambda x: get_color(x, bounds, cmap.colors))

fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

gdf = gpd.read_file('study_extent.shp')

for geometry in gdf['geometry']:
    if isinstance(geometry, Polygon):
        x, y = geometry.exterior.coords.xy
        ax.add_patch(PathPatch(Path(list(zip(x, y))), fill=None, edgecolor='k', linewidth=5))
    elif isinstance(geometry, MultiPolygon):
        for subgeometry in geometry:
            x, y = subgeometry.exterior.coords.xy
            ax.add_patch(PathPatch(Path(list(zip(x, y))), fill=None, edgecolor='k', linewidth=5))

ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle='-', linewidth=2)
ax.gridlines(draw_labels=True, linestyle='--')

sc = ax.scatter(MonthlyStorageChange['lon'], MonthlyStorageChange['lat'], c=MonthlyStorageChange['color'],
                marker='o', s=30, edgecolor=MonthlyStorageChange['color'], transform=ccrs.PlateCarree())

sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

cbar = plt.colorbar(sm, orientation='horizontal', shrink=0.5, ticks=bounds, pad=0.05)
cbar.set_label('Mean Monthly GWS change (mm)', fontsize=15)
cbar.ax.set_xticklabels(['{:.2f}'.format(b) for b in bounds])

ax.set_extent([-10, 49, 25.5, 49])

add_scalebar(ax, 500)

arrow_x, arrow_y = 0.97, .9
ax.text(arrow_x, arrow_y, u'\u25B2\nN', transform=ax.transAxes, ha='center', va='bottom', fontsize=32, fontweight='bold')

plt.title('Spatial Distribution of Mean Monthly Storage Change (GWS-without soil moisture)', fontsize=20, pad=20)
plt.show()


In [ ]:
data = pd.read_csv('GWS_no_soil.csv')
lat_lon = data[['lat', 'lon']]
#lat_lon

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# Calculated the difference in groundwater storage between successive months in each year
data_diff = data.iloc[:, 2:].diff(axis=1)
data_diff

In [ ]:
Ave_Ann = data_diff.iloc[:, 9:].mean(axis=1)

In [ ]:
Ave_Ann = pd.DataFrame(Ave_Ann)

Ave_Ann.columns = ['Ave']

In [ ]:
MonthlyStorageChange = pd.concat([lat_lon, Ave_Ann], axis=1)

In [ ]:
MonthlyStorageChange

In [ ]:
MonthlyStorageChange.to_csv('MonthlyStorageChange_incl_soil.csv', index=None)

In [ ]:
stor_change = 0.48
MonthlyStorageChange_subset = MonthlyStorageChange[MonthlyStorageChange['Ave'] < stor_change]
MonthlyStorageChange_subset

In [ ]:
A, B = MonthlyStorageChange['Ave'].min(), MonthlyStorageChange['Ave'].max()
A, B

In [ ]:


# Function to add a scalebar
def add_scalebar(ax, length, location=(0.5, 0.05), linewidth=4):
    llx0, llx1, lly0, lly1 = ax.get_extent(ccrs.PlateCarree())
    sbllx = (llx1 + llx0) / 2
    sblly = lly0 + (lly1 - lly0) * location[1]
    tmc = ccrs.TransverseMercator(sbllx, sblly)
    x0, x1, y0, y1 = ax.get_extent(tmc)
    sbx = x0 + (x1 - x0) * location[0]
    sby = y0 + (y1 - y0) * location[1]
    bar_xs = [sbx - length * 500, sbx + length * 500]
    ax.plot(bar_xs, [sby, sby], transform=tmc, color='k', linewidth=linewidth)
    ax.text(sbx, sby, str(length) + ' km', transform=tmc,
            horizontalalignment='center', verticalalignment='bottom')

min_value = -2.2681375093774516
max_value = 0.6902278347549021

# Get the absolute maximum value in your data
abs_max = max(abs(min_value), abs(max_value))

# Redefined min_value and max_value to be symmetric around zero
min_value = -abs_max
max_value = abs_max

interval = (max_value - min_value) / 10
bounds = np.linspace(min_value, max_value, 11)
neg_cmap = mcolors.LinearSegmentedColormap.from_list("neg_cmap", ["darkred", "yellow"], 5)
pos_cmap = mcolors.LinearSegmentedColormap.from_list("pos_cmap", ["lightblue", "darkblue"], 5)
cmap = mcolors.ListedColormap(np.concatenate((neg_cmap(np.linspace(0, 1, 5)), pos_cmap(np.linspace(0, 1, 5)))))
norm = mcolors.BoundaryNorm(bounds, cmap.N)

def get_color(value, bounds, colors):
    for i in range(len(bounds) - 1):
        if value >= bounds[i] and value < bounds[i + 1]:
            return colors[i]
    return colors[-1]

MonthlyStorageChange['color'] = MonthlyStorageChange['Ave'].apply(lambda x: get_color(x, bounds, cmap.colors))

fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

gdf = gpd.read_file('study_extent.shp')

for geometry in gdf['geometry']:
    if isinstance(geometry, Polygon):
        x, y = geometry.exterior.coords.xy
        ax.add_patch(PathPatch(Path(list(zip(x, y))), fill=None, edgecolor='k', linewidth=5))
    elif isinstance(geometry, MultiPolygon):
        for subgeometry in geometry:
            x, y = subgeometry.exterior.coords.xy
            ax.add_patch(PathPatch(Path(list(zip(x, y))), fill=None, edgecolor='k', linewidth=5))

ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle='-', linewidth=2)
ax.gridlines(draw_labels=True, linestyle='--')

sc = ax.scatter(MonthlyStorageChange['lon'], MonthlyStorageChange['lat'], c=MonthlyStorageChange['color'],
                marker='o', s=30, edgecolor=MonthlyStorageChange['color'], transform=ccrs.PlateCarree())

sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

cbar = plt.colorbar(sm, orientation='horizontal', shrink=0.5, ticks=bounds, pad=0.05)
cbar.set_label('Mean Monthly GWS change (mm)', fontsize=15)
cbar.ax.set_xticklabels(['{:.2f}'.format(b) for b in bounds])

ax.set_extent([-10, 49, 25.5, 49])

add_scalebar(ax, 500)

arrow_x, arrow_y = 0.97, .9
ax.text(arrow_x, arrow_y, u'\u25B2\nN', transform=ax.transAxes, ha='center', va='bottom', fontsize=32, fontweight='bold')

plt.title('Spatial Distribution of Mean Monthly Storage Change (GWS-including soil moisture)', fontsize=20, pad=20)
plt.show()


In [ ]:
data = pd.read_csv('recharge_with_coord.csv')
lat_lon = data[['lat', 'lon']]
#lat_lon

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# Calculated the difference in groundwater storage between successive months in each year
data_diff = data.iloc[:, 2:].diff(axis=1)
data_diff

In [ ]:
Ave_Ann = data_diff.iloc[:, 9:].mean(axis=1)

In [ ]:
Ave_Ann = pd.DataFrame(Ave_Ann)

Ave_Ann.columns = ['Ave']

In [ ]:
MonthlyStorageChange = pd.concat([lat_lon, Ave_Ann], axis=1)

In [ ]:
MonthlyStorageChange

In [ ]:
MonthlyStorageChange.to_csv('MonthlyStorageChange_Recharge.csv', index=None)

In [ ]:
A, B = MonthlyStorageChange['Ave'].min(), MonthlyStorageChange['Ave'].max()
A, B

In [ ]:

# Function to add a scalebar
def add_scalebar(ax, length, location=(0.5, 0.05), linewidth=4):
    llx0, llx1, lly0, lly1 = ax.get_extent(ccrs.PlateCarree())
    sbllx = (llx1 + llx0) / 2
    sblly = lly0 + (lly1 - lly0) * location[1]
    tmc = ccrs.TransverseMercator(sbllx, sblly)
    x0, x1, y0, y1 = ax.get_extent(tmc)
    sbx = x0 + (x1 - x0) * location[0]
    sby = y0 + (y1 - y0) * location[1]
    bar_xs = [sbx - length * 500, sbx + length * 500]
    ax.plot(bar_xs, [sby, sby], transform=tmc, color='k', linewidth=linewidth)
    ax.text(sbx, sby, str(length) + ' km', transform=tmc,
            horizontalalignment='center', verticalalignment='bottom')

min_value = -0.8426065206985293
max_value =  0.41716603757352944

# Get the absolute maximum value in your data
abs_max = max(abs(min_value), abs(max_value))

# Redefined min_value and max_value to be symmetric around zero
min_value = -abs_max
max_value = abs_max

interval = (max_value - min_value) / 10
bounds = np.linspace(min_value, max_value, 11)
neg_cmap = mcolors.LinearSegmentedColormap.from_list("neg_cmap", ["darkred", "yellow"], 5)
pos_cmap = mcolors.LinearSegmentedColormap.from_list("pos_cmap", ["lightblue", "darkblue"], 5)
cmap = mcolors.ListedColormap(np.concatenate((neg_cmap(np.linspace(0, 1, 5)), pos_cmap(np.linspace(0, 1, 5)))))
norm = mcolors.BoundaryNorm(bounds, cmap.N)

def get_color(value, bounds, colors):
    for i in range(len(bounds) - 1):
        if value >= bounds[i] and value < bounds[i + 1]:
            return colors[i]
    return colors[-1]

MonthlyStorageChange['color'] = MonthlyStorageChange['Ave'].apply(lambda x: get_color(x, bounds, cmap.colors))

fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

gdf = gpd.read_file('study_extent.shp')

for geometry in gdf['geometry']:
    if isinstance(geometry, Polygon):
        x, y = geometry.exterior.coords.xy
        ax.add_patch(PathPatch(Path(list(zip(x, y))), fill=None, edgecolor='k', linewidth=5))
    elif isinstance(geometry, MultiPolygon):
        for subgeometry in geometry:
            x, y = subgeometry.exterior.coords.xy
            ax.add_patch(PathPatch(Path(list(zip(x, y))), fill=None, edgecolor='k', linewidth=5))

ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle='-', linewidth=2)
ax.gridlines(draw_labels=True, linestyle='--')

sc = ax.scatter(MonthlyStorageChange['lon'], MonthlyStorageChange['lat'], c=MonthlyStorageChange['color'],
                marker='o', s=30, edgecolor=MonthlyStorageChange['color'], transform=ccrs.PlateCarree())

sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

cbar = plt.colorbar(sm, orientation='horizontal', shrink=0.5, ticks=bounds, pad=0.05)
cbar.set_label('Mean Monthly Change in Recharge (mm)', fontsize=15)
cbar.ax.set_xticklabels(['{:.2f}'.format(b) for b in bounds])

ax.set_extent([-10, 49, 25.5, 49])

add_scalebar(ax, 500)

arrow_x, arrow_y = 0.97, .9
ax.text(arrow_x, arrow_y, u'\u25B2\nN', transform=ax.transAxes, ha='center', va='bottom', fontsize=32, fontweight='bold')

plt.title('Spatial Distribution of Mean Monthly Change in Recharge', fontsize=20, pad=20)
plt.show()


In [ ]:
data = pd.read_csv('Precipitation_mm.csv')
lat_lon = data[['lat', 'lon']]
#lat_lon

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# Calculated the difference in groundwater storage between successive months in each year
data_diff = data.iloc[:, 2:].diff(axis=1)
data_diff

In [ ]:
Ave_Ann = data_diff.iloc[:, 9:].mean(axis=1)

In [ ]:
Ave_Ann = pd.DataFrame(Ave_Ann)

Ave_Ann.columns = ['Ave']

In [ ]:
MonthlyStorageChange = pd.concat([lat_lon, Ave_Ann], axis=1)

In [ ]:
MonthlyStorageChange

In [ ]:
MonthlyStorageChange.to_csv('MonthlyStorageChange_Recharge.csv', index=None)

In [ ]:
A, B = MonthlyStorageChange['Ave'].min(), MonthlyStorageChange['Ave'].max()
A, B

In [ ]:

# Function to add a scalebar
def add_scalebar(ax, length, location=(0.5, 0.05), linewidth=4):
    llx0, llx1, lly0, lly1 = ax.get_extent(ccrs.PlateCarree())
    sbllx = (llx1 + llx0) / 2
    sblly = lly0 + (lly1 - lly0) * location[1]
    tmc = ccrs.TransverseMercator(sbllx, sblly)
    x0, x1, y0, y1 = ax.get_extent(tmc)
    sbx = x0 + (x1 - x0) * location[0]
    sby = y0 + (y1 - y0) * location[1]
    bar_xs = [sbx - length * 500, sbx + length * 500]
    ax.plot(bar_xs, [sby, sby], transform=tmc, color='k', linewidth=linewidth)
    ax.text(sbx, sby, str(length) + ' km', transform=tmc,
            horizontalalignment='center', verticalalignment='bottom')

min_value = -0.05037670588235293
max_value =  0.04216233333333332

# Got the absolute maximum value in your data
abs_max = max(abs(min_value), abs(max_value))

# Redefined min_value and max_value to be symmetric around zero
min_value = -abs_max
max_value = abs_max

interval = (max_value - min_value) / 10
bounds = np.linspace(min_value, max_value, 11)
neg_cmap = mcolors.LinearSegmentedColormap.from_list("neg_cmap", ["darkred", "yellow"], 5)
pos_cmap = mcolors.LinearSegmentedColormap.from_list("pos_cmap", ["lightblue", "darkblue"], 5)
cmap = mcolors.ListedColormap(np.concatenate((neg_cmap(np.linspace(0, 1, 5)), pos_cmap(np.linspace(0, 1, 5)))))
norm = mcolors.BoundaryNorm(bounds, cmap.N)

def get_color(value, bounds, colors):
    for i in range(len(bounds) - 1):
        if value >= bounds[i] and value < bounds[i + 1]:
            return colors[i]
    return colors[-1]

MonthlyStorageChange['color'] = MonthlyStorageChange['Ave'].apply(lambda x: get_color(x, bounds, cmap.colors))

fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

gdf = gpd.read_file('study_extent.shp')

for geometry in gdf['geometry']:
    if isinstance(geometry, Polygon):
        x, y = geometry.exterior.coords.xy
        ax.add_patch(PathPatch(Path(list(zip(x, y))), fill=None, edgecolor='k', linewidth=5))
    elif isinstance(geometry, MultiPolygon):
        for subgeometry in geometry:
            x, y = subgeometry.exterior.coords.xy
            ax.add_patch(PathPatch(Path(list(zip(x, y))), fill=None, edgecolor='k', linewidth=5))

ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle='-', linewidth=2)
ax.gridlines(draw_labels=True, linestyle='--')

sc = ax.scatter(MonthlyStorageChange['lon'], MonthlyStorageChange['lat'], c=MonthlyStorageChange['color'],
                marker='o', s=30, edgecolor=MonthlyStorageChange['color'], transform=ccrs.PlateCarree())

sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

cbar = plt.colorbar(sm, orientation='horizontal', shrink=0.5, ticks=bounds, pad=0.05)
cbar.set_label('Mean Monthly Change in Precipitation (mm)', fontsize=15)
cbar.ax.set_xticklabels(['{:.2f}'.format(b) for b in bounds])

ax.set_extent([-10, 49, 25.5, 49])

add_scalebar(ax, 500)

arrow_x, arrow_y = 0.97, .9
ax.text(arrow_x, arrow_y, u'\u25B2\nN', transform=ax.transAxes, ha='center', va='bottom', fontsize=32, fontweight='bold')

plt.title('Spatial Distribution of Mean Monthly Change in Precipitation', fontsize=20, pad=20)
plt.show()


In [ ]:
data = pd.read_csv('temp.csv')
lat_lon = data[['lat', 'lon']]
#lat_lon

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# Calculated the difference in groundwater storage between successive months in each year
data_diff = data.iloc[:, 2:].diff(axis=1)
data_diff

In [ ]:
Ave_Ann = data_diff.iloc[:, 9:].mean(axis=1)

In [ ]:
Ave_Ann = pd.DataFrame(Ave_Ann)

Ave_Ann.columns = ['Ave']

In [ ]:
MonthlyStorageChange = pd.concat([lat_lon, Ave_Ann], axis=1)

In [ ]:
MonthlyStorageChange

In [ ]:
MonthlyStorageChange.to_csv('MonthlyStorageChange_Recharge.csv', index=None)

In [ ]:
A, B = MonthlyStorageChange['Ave'].min(), MonthlyStorageChange['Ave'].max()
A, B

In [ ]:

# Function to add a scalebar
def add_scalebar(ax, length, location=(0.5, 0.05), linewidth=4):
    llx0, llx1, lly0, lly1 = ax.get_extent(ccrs.PlateCarree())
    sbllx = (llx1 + llx0) / 2
    sblly = lly0 + (lly1 - lly0) * location[1]
    tmc = ccrs.TransverseMercator(sbllx, sblly)
    x0, x1, y0, y1 = ax.get_extent(tmc)
    sbx = x0 + (x1 - x0) * location[0]
    sby = y0 + (y1 - y0) * location[1]
    bar_xs = [sbx - length * 500, sbx + length * 500]
    ax.plot(bar_xs, [sby, sby], transform=tmc, color='k', linewidth=linewidth)
    ax.text(sbx, sby, str(length) + ' km', transform=tmc,
            horizontalalignment='center', verticalalignment='bottom')

min_value = -0.0045460784313727165
max_value =  0.05411578431372556

# Get the absolute maximum value in your data
abs_max = max(abs(min_value), abs(max_value))

# Redefine min_value and max_value to be symmetric around zero
min_value = -abs_max
max_value = abs_max

interval = (max_value - min_value) / 10
bounds = np.linspace(min_value, max_value, 11)
neg_cmap = mcolors.LinearSegmentedColormap.from_list("neg_cmap", ["darkred", "yellow"], 5)
pos_cmap = mcolors.LinearSegmentedColormap.from_list("pos_cmap", ["lightblue", "darkblue"], 5)
cmap = mcolors.ListedColormap(np.concatenate((neg_cmap(np.linspace(0, 1, 5)), pos_cmap(np.linspace(0, 1, 5)))))
norm = mcolors.BoundaryNorm(bounds, cmap.N)

def get_color(value, bounds, colors):
    for i in range(len(bounds) - 1):
        if value >= bounds[i] and value < bounds[i + 1]:
            return colors[i]
    return colors[-1]

MonthlyStorageChange['color'] = MonthlyStorageChange['Ave'].apply(lambda x: get_color(x, bounds, cmap.colors))

fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

gdf = gpd.read_file('study_extent.shp')

for geometry in gdf['geometry']:
    if isinstance(geometry, Polygon):
        x, y = geometry.exterior.coords.xy
        ax.add_patch(PathPatch(Path(list(zip(x, y))), fill=None, edgecolor='k', linewidth=5))
    elif isinstance(geometry, MultiPolygon):
        for subgeometry in geometry:
            x, y = subgeometry.exterior.coords.xy
            ax.add_patch(PathPatch(Path(list(zip(x, y))), fill=None, edgecolor='k', linewidth=5))

ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle='-', linewidth=2)
ax.gridlines(draw_labels=True, linestyle='--')

sc = ax.scatter(MonthlyStorageChange['lon'], MonthlyStorageChange['lat'], c=MonthlyStorageChange['color'],
                marker='o', s=30, edgecolor=MonthlyStorageChange['color'], transform=ccrs.PlateCarree())

sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

cbar = plt.colorbar(sm, orientation='horizontal', shrink=0.5, ticks=bounds, pad=0.05)
cbar.set_label('Mean Monthly Change in Temperature (Kelvin)', fontsize=15)
cbar.ax.set_xticklabels(['{:.2f}'.format(b) for b in bounds])

ax.set_extent([-10, 49, 25.5, 49])

add_scalebar(ax, 500)

arrow_x, arrow_y = 0.97, .9
ax.text(arrow_x, arrow_y, u'\u25B2\nN', transform=ax.transAxes, ha='center', va='bottom', fontsize=32, fontweight='bold')

plt.title('Spatial Distribution of Mean Monthly Change in Temperature', fontsize=20, pad=20)
plt.show()
